# Lecture 10: Model evaluation & comparison (1)

## Instructor： 胡传鹏（博士）[Dr. Hu Chuan-Peng]

### 南京师范大学心理学院[School of Psychology, Nanjing Normal University]

##  Recap: Linear Model and model diagnositcs

* Workflow
* MCMC diagnostics

我们在上节课中采用一个简化的workflow，具体包括如下几个步骤：



![Image Name](https://cdn.kesci.com/upload/image/rkz1ehen1l.png?imageView2/0/w/960/h/960)




现在，我们通过上节课的例子来简单回顾一下workflow

在本例中，涉及：研究问题、数据收集、选择模型、选择先验、模型拟合、采样过程评估/模型诊断

### (1) 提出研究问题

还有研究发现个体创新行为可能与自尊水平有关。
SES_t: 自尊水平；
EIB_t: 创新行为

### (2) 数据收集

In [1]:
#加载需要使用的库
%matplotlib inline
import numpy as np 
from scipy import stats
import matplotlib.pyplot as plt
import pandas as pd
import arviz as az
import pymc3 as pm
# mpl_toolkits.mplot3d是用于三维画图的，Axes3D用于画三维图
from mpl_toolkits.mplot3d import Axes3D

#数据预处理与可视化
np.random.seed(123)  # 随机数种子，确保随后生成的随机数相同
data = pd.read_csv("/home/mw/input/data9464/clean.csv") # 读取数据，该数据需要提前挂载
data['SES_t'] = (data['SES_t'] - data['SES_t'].mean()) / data['SES_t'].std()#将变量进行标准化

data['EIB_t'] = (data['EIB_t'] - data['EIB_t'].mean()) / data['EIB_t'].std()#将变量进行标准化

plt.scatter(data['SES_t'],data['EIB_t'])
plt.xlabel('SES_t')
plt.ylabel('EIB_t')

WARNING (theano.link.c.cmodule): install mkl with `conda install mkl-service`: No module named 'mkl'


Text(0, 0.5, 'EIB_t')

<Figure size 432x288 with 1 Axes>

### (3) 模型设定

尝试构建一个简单的线性模型

线性模型可以用概率的形式进行表达


$\alpha \sim Normal(0,1)$      ->      a $\sim$ Normal(mu,sigma)

$\beta  \sim Normal(0,1)$      ->      b $\sim$ Normal(mu,sigma)

$\sigma \sim HalfNormal(1)$      ->      sigma $\sim$ HalfNormal(1)

$\mu_i  = \alpha + \beta *x$      ->      mu = alpha + beta*x 

$y \sim Normal(\mu_i,sigma)$      ->      y $\sim$ Normal(mu,sigma)

### (4) 选择先验

In [2]:
# 在pymc3中，pm.Model()定义了一个新的模型对象，这个对象是模型中随机变量的容器
# 在python中，容器是一种数据结构，是用来管理特殊数据的对象
# with语句定义了一个上下文管理器，以 linear_model 作为上下文，在这个上下文中定义的变量都被添加到这个模型
with pm.Model() as linear_model:
    # 先验分布: alpha, beta, sigma这三个参数是随机变量
    alpha = pm.Normal('alpha',mu=0,sd=1)
    beta = pm.Normal('beta',mu=0,sd=1, shape=1)  
    sigma = pm.HalfNormal('sigma',sd=1)
    
    # 先验预测检查
    prior_checks = pm.sample_prior_predictive(samples=50)

In [3]:
fig, ax = plt.subplots()
x1 = np.linspace(-3, 3, 50) # 生成从-2，2之间的50个假数据

for a, b in zip(prior_checks["alpha"], prior_checks["beta"]):
    y = a + b * x1          # 基于假数据生成预测值
    ax.plot(x1,y)

<Figure size 432x288 with 1 Axes>

### (5) 拟合数据

In [4]:
linear_model = pm.Model()
with linear_model :
    # 在pymc3中，pm.Model()定义了一个新的模型对象，这个对象是模型中随机变量的容器
    # 在python中，容器是一种数据结构，是用来管理特殊数据的对象
    # with语句定义了一个上下文管理器，以 linear_model 作为上下文，在这个上下文中定义的变量都被添加到这个模型
    alpha = pm.Normal('alpha', mu=0, sd=1)
    beta = pm.Normal('beta', mu=0, sd=1, shape=1)
    sigma = pm.HalfNormal('sigma', sd=1)
    # x为自变量，是之前已经载入的数据
    x = pm.Data("x", data['SES_t'])
    # 线性模型：mu是确定性随机变量，这个变量的值完全由右端值确定
    mu = pm.Deterministic("mu", alpha + beta*x) 
    # Y的观测值，这是一个特殊的观测随机变量，表示模型数据的可能性。也可以表示模型的似然，通过 observed 参数来告诉这个变量其值是已经被观测到了的，不会被拟合算法改变
    y_obs = pm.Normal('y_obs', mu=mu, sd=sigma, observed=data['EIB_t'])

### (6) 采样过程诊断

如果使用MCMC对后验进行近似，则需要首先对MCMC过程进行评估。

* 是否收敛；
* 是否接近真实的后验。

对采样过程的评估我们会采用目视检查或rhat这个指标

In [5]:
with linear_model :
    # 使用mcmc方法进行采样，draws为采样次数，tune为调整采样策略的次数，这些次数将在采样结束后被丢弃，
    # target_accept为接受率， return_inferencedata=True为该函数返回的对象是arviz.InnferenceData对象
    # chains为我们采样的链数，cores为我们的调用的cpu数，多个链可以在多个cpu中并行计算，我们在和鲸中调用的cpu数为2
    trace = pm.sample(draws=2000, tune=1000, target_accept=0.9, chains=2, cores= 2, return_inferencedata=True)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (2 chains in 2 jobs)
NUTS: [sigma, beta, alpha]


Sampling 2 chains for 1_000 tune and 2_000 draw iterations (2_000 + 4_000 draws total) took 6 seconds.


In [6]:
az.plot_trace(trace,var_names=['alpha','beta','sigma'])

array([[<AxesSubplot:title={'center':'alpha'}>,
        <AxesSubplot:title={'center':'alpha'}>],
       [<AxesSubplot:title={'center':'beta'}>,
        <AxesSubplot:title={'center':'beta'}>],
       [<AxesSubplot:title={'center':'sigma'}>,
        <AxesSubplot:title={'center':'sigma'}>]], dtype=object)

<Figure size 864x432 with 6 Axes>

In [7]:
az.summary(trace, var_names=['alpha','beta','sigma'], kind="diagnostics")

,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
alpha,0.0,0.0,5018.0,2745.0,1.0
beta[0],0.0,0.0,4503.0,3309.0,1.0
sigma,0.0,0.0,4367.0,3085.0,1.0


### (7) 模型诊断

在MCMC有效的前提下，需要继续检验模型是否能够较好地拟合数据。

我们会使用后验预测分布通过我们得到的参数生成一批模拟数据，并将其与真实数据进行对比。

In [8]:
# 后验预测分布的计算仍在容器中进行
with linear_model:
    # pm.sample_posterior_predictive()利用trace.posterior的后验分布计算后验预测分布
    ppc_y = pm.sample_posterior_predictive(trace.posterior) 
#将ppc_y转化为InferenceData对象合并到trace中
az.concat(trace, az.from_pymc3(posterior_predictive=ppc_y), inplace=True)

/opt/conda/lib/python3.7/site-packages/arviz/data/io_pymc3.py:100: FutureWarning: Using `from_pymc3` without the model will be deprecated in a future release. Not using the model will return less accurate and less useful results. Make sure you use the model argument or call from_pymc3 within a model context.
  FutureWarning,


In [9]:
# 绘制后验预测分布
az.plot_ppc(trace)

<AxesSubplot:xlabel='y_obs'>

/opt/conda/lib/python3.7/site-packages/IPython/core/events.py:89: UserWarning: Creating legend with loc="best" can be slow with large amounts of data.
  func(*args, **kwargs)


<Figure size 432x288 with 1 Axes>

## Part 2: 模型评估/比较与选择(Model evaluation/comparison & selection)

我们在上节课中提到的workflow具体包括如下几个步骤：

研究问题、选择模型、选择先验、模型拟合、采样过程评估、模型评估、模型比较、统计推断、结果报告。


![Image Name](https://cdn.kesci.com/upload/image/rkz1dqyo2e.png?imageView2/0/w/960/h/960)

在贝叶斯的workflow中，MCMC评估是对MCMC进行检验，在Gelman et al (2020)中也被称为计算过程的验证(validate computation)，其核心在于确定MCMC算法采样得到的样本是否足以提供目标分布的精确近似。

模型评估则是指对模型是否有效/可信进行评估，既可以是对单个模型，也可以是对多个模型进行。

模型评估的核心在于模型捕捉到了数据中的关键模式，既非太简单而错过数据中有价值的信息(**欠拟合, underfitting**)，也不会太复杂从而将数据中的噪音加入到模型中(**过拟合, overfitting**)。


![Image Name](https://img-blog.csdnimg.cn/20210303141100499.png#pic_center)

资料来源：https://blog.csdn.net/weixin_43378396/article/details/90707493

### 欠拟合(underfitting)

* 欠拟合的模型在当前样本的数据拟合效果不好，且其泛化能力(模型在当前样本外新的数据上的预测的准确度)也同样不佳。

### 过拟合(overfitting)

* 模型在当前样本的数据上的拟合程度极好，但是泛化能力也较差。

* 模型把训练样本学习地“太好了”，把样本自身地一些噪音也当作了所有潜在样本都会具有的一些性质，这样就会导致其泛化性能下降。


### 为什么会发生欠拟合

- 数据特征较少

数据特征指的是数据的属性，比如第一部分中展示的数据的各个变量就是数据的特征。在所有变量都能独立地对目标变量做出解释的前提下，数据特征越多，数据拟合程度越好。

- 模型复杂度过低

模型的复杂度代表模型能够描述的所有函数，比如线性回归最多能表示所有的线性函数。

模型的复杂度和模型的参数数量有关，一般来说，模型参数越多，复杂度越高，模型参数越少，复杂度越低。

### 为什么会发生过拟合

- 当前样本的噪音过大，模型将噪音当作数据本身的特征

当数据的有些特征与目标变量无关，这些特征就是噪音，但它也可能被误当作数据特征，这就会造成模型过拟合

- 样本选取有误，样本不能代表整体

- 模型参数太多，模型复杂度太高



![Image Name](https://vitalflux.com/wp-content/uploads/2020/12/overfitting-and-underfitting-wrt-model-error-vs-complexity-768x443.png)

资料来源：https://vitalflux.com/overfitting-underfitting-concepts-interview-questions/

### 如何避免欠拟合

- 增加数据的特征

- 增加模型复杂度

### 如何避免过拟合

- 选择更具代表性的数据

- 降低模型复杂度


**问题的本质在于：模型与数据真实的生成模型匹配**